# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [86]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [87]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,25.32,69,27,7.19,YE,1701463667
1,1,avarua,-21.2078,-159.7750,27.03,86,100,8.23,CK,1701463667
2,2,blackmans bay,-43.0167,147.3167,12.63,90,100,1.07,AU,1701463668
3,3,waitangi,-43.9535,-176.5597,13.23,72,46,2.24,NZ,1701463668
4,4,weno,7.4515,151.8468,26.97,89,75,1.54,FM,1701463668


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [88]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.50,
)

# Display the mapcomd
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [89]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 18) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] > 5) &
    (city_data_df["Wind Speed"] < 7) &
    (city_data_df["Cloudiness"] > 20) &
    (city_data_df["Cloudiness"] < 80)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,broken hill,-31.9500,141.4333,18.14,56,71,5.66,AU,1701463676
66,66,tsiombe,-25.3000,45.4833,19.89,80,22,5.19,MG,1701463683
136,136,ribeira grande,38.5167,-28.7000,18.22,72,75,6.69,PT,1701463706
143,143,touros,-5.1989,-35.4608,26.31,76,32,5.49,BR,1701463698
160,160,crane,31.3974,-102.3501,18.38,29,59,5.70,US,1701463713
185,185,paramonga,-10.6667,-77.8333,22.17,75,65,5.19,PE,1701463720
248,248,urangan,-25.3000,152.9000,24.95,88,62,5.77,AU,1701463735
269,269,villa vicente guerrero,19.1194,-98.1708,21.66,37,41,5.02,MX,1701463740
332,332,sherwood,34.8151,-92.2243,20.12,65,75,5.14,US,1701463761
442,442,sierra grande,-41.6060,-65.3557,22.86,28,62,6.24,AR,1701463793


### Step 3: Create a new DataFrame called `hotel_df`.

In [90]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
38,broken hill,AU,-31.9500,141.4333,56,
66,tsiombe,MG,-25.3000,45.4833,80,
136,ribeira grande,PT,38.5167,-28.7000,72,
143,touros,BR,-5.1989,-35.4608,76,
160,crane,US,31.3974,-102.3501,29,
185,paramonga,PE,-10.6667,-77.8333,75,
248,urangan,AU,-25.3000,152.9000,88,
269,villa vicente guerrero,MX,19.1194,-98.1708,37,
332,sherwood,US,34.8151,-92.2243,65,
442,sierra grande,AR,-41.6060,-65.3557,28,


In [91]:
# Make an API request using the params dictionary
response = requests.get(base_url, params=params)

# Print the API response content for debugging
print(response.text)

# Convert the API response to JSON format
name_address = response.json()

{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Hotel Regina Resort & Convenciones","country":"Bolivia","country_code":"bo","state":"Cochabamba","county":"Quillacollo","city":"Apote","district":"Tinti Moqho","street":"Avenida Cochabamba","lon":-66.23070086161417,"lat":-17.33407745,"state_code":"C","formatted":"Hotel Regina Resort & Convenciones, Avenida Cochabamba, Apote, Bolivia","address_line1":"Hotel Regina Resort & Convenciones","address_line2":"Avenida Cochabamba, Apote, Bolivia","categories":["accommodation","accommodation.hotel","internet_access","internet_access.for_customers"],"details":["details","details.accommodation","details.contact","details.facilities","details.payment"],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Licence","url":"https://www.openstreetmap.org/copyright","raw":{"name":"Hotel Regina Resort & Convenciones","email":"resort@hotelreginabolivia.com","phone":"+5

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
broken hill - nearest hotel: Ibis Styles
tsiombe - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
touros - nearest hotel: Pousada Atlântico
crane - nearest hotel: No hotel found
paramonga - nearest hotel: Hotel Rico
urangan - nearest hotel: Ramada
villa vicente guerrero - nearest hotel: HOTEL SNAYLF
sherwood - nearest hotel: Wingate By Wyndham North Little Rock
sierra grande - nearest hotel: Sierra Grande
callao - nearest hotel: Casa Ronald
pandan - nearest hotel: No hotel found
rawai - nearest hotel: Rawai Boutique Resort
tiquipaya - nearest hotel: Hotel Regina Resort & Convenciones


,City,Country,Lat,Lng,Humidity,Hotel Name
38,broken hill,AU,-31.9500,141.4333,56,Ibis Styles
66,tsiombe,MG,-25.3000,45.4833,80,No hotel found
136,ribeira grande,PT,38.5167,-28.7000,72,Hospedaria JSF
143,touros,BR,-5.1989,-35.4608,76,Pousada Atlântico
160,crane,US,31.3974,-102.3501,29,No hotel found
185,paramonga,PE,-10.6667,-77.8333,75,Hotel Rico
248,urangan,AU,-25.3000,152.9000,88,Ramada
269,villa vicente guerrero,MX,19.1194,-98.1708,37,HOTEL SNAYLF
332,sherwood,US,34.8151,-92.2243,65,Wingate By Wyndham North Little Rock
442,sierra grande,AR,-41.6060,-65.3557,28,Sierra Grande


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    alpha=0.50,
    hover_cols=["City", "Hotel Name", "Country"],
)
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)